[Reference](https://medium.com/@Rohan_Dutt/10-bayesian-optimization-tricks-for-hyperparameters-in-ml-you-can-use-today-7a149a32c830)

# Use Priors Like a Bayesian Pro

In [4]:
!pip install scikit-optimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 4.8 MB/s eta 0:00:00


In [6]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from skopt import Optimizer


# Step 1: Quick cheap search to build priors
def objective(params):
    lr, depth = params
    return train_model(lr, depth)  # your training loop returning validation loss
search_space = [
    (1e-4, 1e-1),   # learning rate
    (2, 10)         # depth
]
# quick 8-run grid/random search
initial_points = [
    (1e-4, 4), (1e-3, 4), (1e-2, 4),
    (1e-4, 8), (1e-3, 8), (1e-2, 8),
    (5e-3, 6), (8e-3, 10)
]
initial_results = [objective(p) for p in initial_points]
# Step 2: Build priors for Bayesian Optimization
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, normalize_y=True)
# Step 3: Initialize optimizer with priors
opt = Optimizer(
    dimensions=search_space,
    base_estimator=gp,
    initial_point_generator="sobol",
)
# Feed prior observations
for p, r in zip(initial_points, initial_results):
    opt.tell(p, r)
# Step 4: Bayesian Optimization with informed priors
for _ in range(30):
    next_params = opt.ask()
    score = objective(next_params)
    opt.tell(next_params, score)
best_params = opt.get_result().x
print("Best Params:", best_params)

# Dynamically Adjust Your Acquisition Function

In [8]:
import numpy as np
from skopt import Optimizer
from skopt.acquisition import gaussian_ei, gaussian_pi, gaussian_ucb

# Dummy expensive objective
def objective(params):
    lr, depth = params
    return train_model(lr, depth)  # Replace with your actual training loop
space = [(1e-4, 1e-1), (2, 10)]
opt = Optimizer(
    dimensions=space,
    base_estimator="GP",
    acq_func="EI"   # initial acquisition function
)
def should_switch(iteration, recent_scores):
    # Simple heuristic: if scores haven't improved in last 5 steps, switch mode
    if iteration > 10 and np.std(recent_scores[-5:]) < 1e-4:
        return True
    return False
scores = []
for i in range(40):
    # Dynamically pick acquisition function
    if should_switch(i, scores):
        # Choose UCB when nearing convergence, PI for risky exploration
        opt.acq_func = "UCB" if scores[-1] < np.median(scores) else "PI"
    x = opt.ask()
    y = objective(x)
    scores.append(y)
    opt.tell(x, y)
best_params = opt.get_result().x
print("Best Params:", best_params)

# Early Stopping for Hyper-hypers

In [10]:
import numpy as np
from skopt import Optimizer
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel

# Meta-learned priors from previous similar tasks
meta_length_scale = 0.3
meta_noise_level = 1e-3
kernel = (
    Matern(length_scale=meta_length_scale, nu=2.5) +
    WhiteKernel(noise_level=meta_noise_level)
)
# Early-stop BO's own hyperparameter tuning
gp = GaussianProcessRegressor(
    kernel=kernel,
    optimizer="fmin_l_bfgs_b",
    n_restarts_optimizer=0,    # Crucial: prevent expensive hyper-hyper loops
    normalize_y=True
)
# BO with a stable, meta-initialized GP
opt = Optimizer(
    dimensions=[(1e-4, 1e-1), (2, 12)],
    base_estimator=gp,
    acq_func="EI"
)
def objective(params):
    lr, depth = params
    return train_model(lr, depth)   # your model's validation loss
scores = []
for _ in range(40):
    x = opt.ask()
    y = objective(x)
    opt.tell(x, y)
    scores.append(y)
best_params = opt.get_result().x
print("Best Params:", best_params)

# Penalize Promising but Costly Regions

In [11]:
import numpy as np
from skopt import Optimizer
from skopt.acquisition import gaussian_ei

# Objective returns BOTH validation loss and estimated training cost
def objective(params):
    lr, depth = params
    val_loss = train_model(lr, depth)
    cost = estimate_cost(lr, depth)   # e.g., GPU hours or FLOPs proxy
    return val_loss, cost
# Custom cost-aware EI: maximize EI / Cost
def cost_aware_ei(model, X, y_min, costs):
    raw_ei = gaussian_ei(X, model, y_min=y_min)
    normalized_costs = costs / np.max(costs)
    penalty = 1.0 / (1e-6 + normalized_costs)
    return raw_ei * penalty
# Search space
opt = Optimizer(
    dimensions=[(1e-4, 1e-1), (2, 20)],
    base_estimator="GP"
)
observed_losses = []
observed_costs = []
for _ in range(40):
    # Ask a batch of candidate points
    candidates = opt.ask(n_points=20)

    # Evaluate cost-aware EI for each candidate
    y_min = np.min(observed_losses) if observed_losses else np.inf
    cost_scores = cost_aware_ei(
        opt.base_estimator_,
        np.array(candidates),
        y_min=y_min,
        costs=np.array(observed_costs[-len(candidates):] + [1]*len(candidates))  # fallback cost=1
    )
    # Pick best candidate under cost-awareness
    next_x = candidates[np.argmax(cost_scores)]

    (loss, cost) = objective(next_x)

    observed_losses.append(loss)
    observed_costs.append(cost)

    opt.tell(next_x, loss)
best_params = opt.get_result().x
print("Best Params (Cost-Aware):", best_params)

# Hybridize BO with Random Search

In [12]:
import numpy as np
from skopt import Optimizer
from skopt.space import Real, Integer

# Define search space
space = [
    Real(1e-4, 1e-1, name="lr"),
    Integer(2, 12, name="depth")
]
# Expensive training loop
def objective(params):
    lr, depth = params
    return train_model(lr, depth)   # your model's validation loss
# BO Optimizer
opt = Optimizer(
    dimensions=space,
    base_estimator="GP",
    acq_func="EI"
)
n_total = 50
n_random = int(0.20 * n_total)      # first 20% = random exploration
results = []
for i in range(n_total):
    if i < n_random:
        # ----- Phase 1: Pure Random Search -----
        x = [
            np.random.uniform(1e-4, 1e-1),
            np.random.randint(2, 13)
        ]
    else:
        # ----- Phase 2: Bayesian Optimization -----
        x = opt.ask()
    y = objective(x)
    results.append((x, y))
    # Only tell BO after evaluations (keeps history consistent)
    opt.tell(x, y)
best_params = opt.get_result().x
# print("Best Params (Hybrid):", best_params)a